# GUC NETW 1003 Kmeans Assginmnet 

Objective: exercise is to learn how to use build and use  kmeans clustering algortrithm    

At the end of this lab you will be able to
> 1. Build and run a K-means algorithm.
> 2. Know how to evalute K-means Algorithm 
> 2. Understand the effect of the parmeters ont he perfromance of the algorithm .
> 3. Understand the effect of distance.

## 1. Wirte K means function 

import required libraries 

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons

> ### 1.1. Code for the distance function 

write a function to calculate the distance between a set of cluster centroids and all the data points 

Cluster_Distances = GUC_Distance ( Cluster_Centroids, Data_points, Distance_Type) 
* The function shoud be able to handel data points with any number of features and any number of points 
* The function should be able to handle any K number of cluster centroids 
* To inrease speed do that in matrix format using numpy
* This should produce a matrix with rows equal to data points columns equal K 
* You should be able to calculate different types of Distance. In this assginmnet you are required to implement the following two 
    - Ecluidian distance
    - Pearson correlation distance


In [ ]:
def GUC_Distance(Cluster_Centroids, Data_points, Distance_Type):

    num_clusters = Cluster_Centroids.shape[0]
    num_points = Data_points.shape[0]

    Cluster_Distance = np.zeros((num_points, num_clusters))

    if Distance_Type == 'euclidean':
        for i in range(num_clusters):
            centroid = Cluster_Centroids.values[i]
            distances = np.sqrt(np.sum((Data_points - centroid)**2, axis=1))
            Cluster_Distance[:, i] = distances

    elif Distance_Type == 'pearson':
            mean_points = np.mean(Data_points, axis=0)
            mean_centroids = np.mean(Cluster_Centroids, axis=0)
            new_points = Data_points - mean_points
            for i in range(num_clusters):
                centroid = Cluster_Centroids.values[i]
                new_centroid = centroid - mean_centroids
                numerator = np.sum(new_points * new_centroid, axis=1)
                denominator_points = np.sqrt(np.sum(new_points ** 2, axis=1))
                denominator_centroid = np.sqrt(np.sum(new_centroid ** 2))
                denominator = denominator_points * denominator_centroid
                denominator[denominator == 0] = np.inf
                Cluster_Distance[:, i] = 1 - (numerator / denominator)

    return Cluster_Distance

> ### 1.2. Code for K means function 

> Loop until a stopping condition 

> > in the loop do the following 
> > 1. Cluster Assginmnet 
> > > * Use the GUC_Distance to calculate between each cluster head and all the points 
> > > * Find the value and index of the minimum distance  for each row. 
> > > * for each point in the row use the index of the minmum distance to represent the cluster Centroid that this point will belong to.
> > > * Calculate and record the mean square distance for each cluster  
> > > * Use the mean square distances to calculate a stopping condition (e.g the sum of mean distances)
> > 2. Updtate Centroids  
> > > * Update the value for the cluster heads by calculating the mean value for the points in each cluster 
> > >  * Calculate the Cluster Metric (Distortion Function)
> > > > a number indicating the how good the clusters are . An example cluster metric is the  (sum of squared error between each point and its cluster center)

In [ ]:
def GUC_Kmean(Data_points, Number_of_Clusters, Distance_Type):

    data_ranges = np.ptp(Data_points.values, axis=0)
    cluster_heads = np.random.rand(Number_of_Clusters, Data_points.shape[1]) * data_ranges
    cluster_heads_df = pd.DataFrame(cluster_heads, columns=Data_points.columns)
    km = []

    # Stopping condition parameters
    min_distance_change = 0.0001
    prev_mean_square_distances = np.inf
    prev_sum_of_distances = np.inf

    Cluster_Metric =0

    while True:
        # Cluster Assignment
        Final_Cluster_Distance = GUC_Distance(cluster_heads_df, Data_points, Distance_Type)
        cluster_assignments = np.argmin(Final_Cluster_Distance, axis=1)

        # Mean square distance for each cluster
        sum_of_distances=0
        mean_square_distances = np.zeros(Number_of_Clusters)
        for i in range(Number_of_Clusters):
            cluster_points= Data_points.iloc[cluster_assignments==i]
            mean_square_distances[i]= np.mean(np.sum((cluster_points-cluster_heads[i])**2, axis=1))
            sum_of_distances+=np.sum((cluster_points - cluster_heads[i])**2, axis=0)


        # Stopping condition

        if np.sum(mean_square_distances) < min_distance_change or np.allclose(mean_square_distances, prev_mean_square_distances):
            break
        eps= 0.0001
        if (abs(sum_of_distances - prev_sum_of_distances) < eps).any():
            break
        prev_mean_square_distances = mean_square_distances.copy()
        prev_sum_of_distances = sum_of_distances

        # Update Centroids
        for i in range(Number_of_Clusters):
            cluster_points = Data_points.iloc[cluster_assignments == i]
            if len(cluster_points) > 0:
                cluster_heads[i] = np.mean(cluster_points, axis=0)

        km.append((cluster_assignments, cluster_heads))

    # Calculate Cluster Metric (Distortion Function)
    Cluster_Metric += np.sum(sum_of_distances)
    display_clusters(Data_points, km,Number_of_Clusters)

    return [Final_Cluster_Distance, Cluster_Metric]

## 2. Display Results 

* below is a an example fuction that displayes Clusters in 2D 

* You need to change the function so that is can display an n dimension data in the form of pairs of subplot 2D figures 

* The function is a helper function that is originaly be found in 
https://www.coursera.org/learn/ibm-unsupervised-learning/


In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

> #### 2.1. helper Display function

In [ ]:
# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_clusters(X, km, num_clusters):
    fig, axs = plt.subplots(X.shape[1], X.shape[1], figsize=(30, 30))
    color = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'brown', 'teal', 'purple']  # List of colors #List of colors
    alpha = 0.5 #color obaque
    s = 50

    for dim_i, dim_1 in enumerate(X.columns):
        for dim_j, dim_2 in enumerate(X.columns):
            for i in range(num_clusters):
                if km and km[-1]:
                    assigned_indices = np.where(km[-1][0] == i)[0]
                    axs[dim_i, dim_j].scatter(X.iloc[assigned_indices, dim_i], X.iloc[assigned_indices, dim_j],
                                              c=color[i], alpha=alpha, s=s, marker='o')
                    axs[dim_i, dim_j].scatter(km[-1][1][i, dim_i], km[-1][1][i, dim_j],
                                              c=color[i], marker='x', s=100)

> #### 2.2. Example 1 : Circular Data Gen and display 

* below is an example of how to use the funciton 
  *  prepare the figure size and background 
   > * this part can be replaced by a number of subplots 
  
  * Produce a data set that represent the x and y o coordinates of a circle 
   > * this part can be replaced by data that you import froma file 
  
  * Data is displayed 
   > * to display the data only it is assumed that the number of clusters is zero which is the default of the fuction 

In [ ]:
# prepare the figure sise and background 
# this part can be replaced by a number of subplots 
plt.rcParams['figure.figsize'] = [8,8]
sns.set_style("whitegrid")
sns.set_context("talk")
# Produce a data set that represent the x and y o coordinates of a circle 
# this part can be replaced by data that you import froma file 
angle = np.linspace(0,2*np.pi,20, endpoint = False)
X = np.append([np.cos(angle)],[np.sin(angle)],0).transpose()
# Data is displayed 
# to display the data only it is assumed that the number of clusters is zero which is the default of the fuction 
display_cluster(X)

> #### 2.3. Example 2 : Multi blob Data Gen and display 

In [ ]:
n_samples = 1000
n_bins = 4  
centers = [(-3, -3), (0, 0), (3, 3), (6, 6), (9,9)]
X, y = make_blobs(n_samples=n_samples, n_features=2, cluster_std=1.0,
                  centers=centers, shuffle=False, random_state=42)
display_cluster(X)

#### 2.3. Example 3 : moons Data Gen and display 

In [ ]:
n_samples = 1000
X, y = noisy_moons = make_moons(n_samples=n_samples, noise= .1)
display_cluster(X)

## 3. Testing the GUC_Kmean function on Simple 2D
1. Apply the GUC_Kmean fucntion to 2D Data provided by example 1, Example 2 and Example 3
2. Try a varing number of cluster centriods that range from 2 to 10 clusters on each set 
3. Display your results.
4. plot the Cluster perfromance Metric versus the number of clusters
4. Change the distance function and repeat the above steps

In [ ]:
#ex.1 - 'euclidean'
plt.rcParams['figure.figsize'] = [8,8]
sns.set_style("whitegrid")
sns.set_context("talk")

angle = np.linspace(0, 2 * np.pi, 20, endpoint=False)
X = np.append([np.cos(angle)], [np.sin(angle)], 0).transpose()

X= pd.DataFrame(X, columns=['X', 'Y'])

cluster_metrics1 = []
clusters_list1= []

for k in range(2, 10):
    Number_of_Clusters = k
    Distance_Type= 'euclidean'
    Final_Cluster_Distance, Cluster_Metric=GUC_Kmean(X, Number_of_Clusters, Distance_Type)
    cluster_metrics1.append(Cluster_Metric)
    clusters_list1.append(Number_of_Clusters)


In [ ]:
# Plot the Cluster Performance Metric versus the Number of Clusters
plt.plot(clusters_list1, cluster_metrics1, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
#ex.2 - 'euclidean'
n_samples = 1000
n_bins = 4
centers = [(-3, -3), (0, 0), (3, 3), (6, 6), (9, 9)]
X, y = make_blobs(n_samples=n_samples, n_features=2, cluster_std=1.0,
                  centers=centers, shuffle=False, random_state=42)
X = pd.DataFrame(X, columns=['X', 'Y'])

clusters_list2 = []
cluster_metrics2 = []

for k in range(2,10):
    Number_of_Clusters = k
    Distance_Type = 'euclidean'
    Final_Cluster_Distance, Cluster_Metric=GUC_Kmean(X, Number_of_Clusters,Distance_Type)
    cluster_metrics2.append(Cluster_Metric)
    clusters_list2.append(Number_of_Clusters)

In [ ]:
# Plot the Cluster Performance Metric versus the Number of Clusters
plt.plot(clusters_list2, cluster_metrics2, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
#ex.3 - 'euclidean'

n_samples = 1000
X, y = make_moons(n_samples=n_samples, noise=0.1)

X = pd.DataFrame(X, columns=['X', 'Y'])

clusters_list3 = []
cluster_metrics3 = []

for k in range(2,10):
    Number_of_Clusters = k
    Distance_Type = 'euclidean'
    Final_Cluster_Distance, Cluster_Metric = GUC_Kmean(X, Number_of_Clusters, Distance_Type)

    cluster_metrics3.append(Cluster_Metric)
    clusters_list3.append(Number_of_Clusters)

In [ ]:
# Plot the Cluster Performance Metric versus the Number of Clusters
plt.plot(clusters_list3, cluster_metrics3, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
#ex.1 - 'pearson'

plt.rcParams['figure.figsize'] = [8,8]
sns.set_style("whitegrid")
sns.set_context("talk")

angle = np.linspace(0, 2 * np.pi, 20, endpoint=False)
X = np.append([np.cos(angle)], [np.sin(angle)], 0).transpose()

X= pd.DataFrame(X, columns=['X', 'Y'])

cluster_metrics1 = []
clusters_list1= []

for k in range(2, 10):
    Number_of_Clusters = k
    Distance_Type= 'pearson'
    Final_Cluster_Distance, Cluster_Metric=GUC_Kmean(X, Number_of_Clusters, Distance_Type)
    cluster_metrics1.append(Cluster_Metric)
    clusters_list1.append(Number_of_Clusters)

In [ ]:
plt.plot(clusters_list1, cluster_metrics1, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
#ex.2 - 'pearson'
n_samples = 1000
n_bins = 4
centers = [(-3, -3), (0, 0), (3, 3), (6, 6), (9, 9)]
X, y = make_blobs(n_samples=n_samples, n_features=2, cluster_std=1.0,
                  centers=centers, shuffle=False, random_state=42)
X = pd.DataFrame(X, columns=['X', 'Y'])

clusters_list2 = []
cluster_metrics2 = []

for k in range(2,10):
    Number_of_Clusters = k
    Distance_Type = 'pearson'
    Final_Cluster_Distance, Cluster_Metric=GUC_Kmean(X, Number_of_Clusters,Distance_Type)
    cluster_metrics2.append(Cluster_Metric)
    clusters_list2.append(Number_of_Clusters)

In [ ]:
# Plot the Cluster Performance Metric versus the Number of Clusters
plt.plot(clusters_list2, cluster_metrics2, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
#ex.3 - 'pearson'

n_samples = 1000
X, y = make_moons(n_samples=n_samples, noise=0.1)

X = pd.DataFrame(X, columns=['X', 'Y'])

clusters_list3 = []
cluster_metrics3 = []

for k in range(2,10):
    Number_of_Clusters = k
    Distance_Type = 'pearson'
    Final_Cluster_Distance, Cluster_Metric = GUC_Kmean(X, Number_of_Clusters, Distance_Type)

    cluster_metrics3.append(Cluster_Metric)
    clusters_list3.append(Number_of_Clusters)


In [ ]:
# Plot the Cluster Performance Metric versus the Number of Clusters
plt.plot(clusters_list3, cluster_metrics3, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

## 4. Using the GUC_Kmean function to segment customers int he provided customer dataset
1. Apply the GUC_Kmean fucntion to customer data  
2. Try a varing number of cluster that range from 2 to 10 cluster on each set 
3. Display your results.
4. plot the Cluster Metric versus the number of clusters
4. Change the distance function and repeat the above steps 

In [ ]:
data = pd.read_csv('C:/Users/Adham/Desktop/Customer data.csv')

In [ ]:
data.set_index('ID', inplace=True)

In [ ]:
def Kmean_initialization(df, Number_of_Clusters):
    initial_centroids= df.iloc[np.random.choice(len(df), size=Number_of_Clusters, replace=False)]
    return initial_centroids

In [ ]:
# 'euclidean'

cluster_metrics_e= []
clusters_list_e=[]

for k in range (2,10):
    Number_of_Clusters= k
    Distance_Type= 'euclidean'
    Cluster_Centroids= Kmean_initialization(data, Number_of_Clusters)
    Final_Cluster_Distance, Cluster_Metric= GUC_Kmean(data, Number_of_Clusters, Distance_Type)
    cluster_metrics_e.append(Cluster_Metric)
    clusters_list_e.append(Number_of_Clusters)

plt.plot(clusters_list_e, cluster_metrics_e, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
# 'pearson'

cluster_metrics_p= []
clusters_list_p=[]

for k in range (2,10):
    Number_of_Clusters= k
    Distance_Type= 'pearson'
    Cluster_Centroids= Kmean_initialization(data, Number_of_Clusters)
    Final_Cluster_Distance, Cluster_Metric= GUC_Kmean(data, Number_of_Clusters, Distance_Type)
    cluster_metrics_p.append(Cluster_Metric)
    clusters_list_p.append(Number_of_Clusters)

plt.plot(clusters_list_p, cluster_metrics_p, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cluster Performance Metric')
plt.title('Cluster Performance Metric vs. Number of Clusters')
plt.grid(True)
plt.show()